In [1]:
import traceback
import os
import json
import time
import ml_collections
import lightning as L
import torch
from lightning.pytorch.callbacks import ModelCheckpoint, LearningRateMonitor
from lightning.pytorch import seed_everything
from lightning.pytorch.loggers import WandbLogger
from seisLM.model.foundation.pretrained_models import LitMultiDimWav2Vec2
from seisLM.data_pipeline import collator
from seisLM.data_pipeline import seisbench_dataloaders as dataloaders
from seisLM.utils import project_path
from seisLM.utils.wandb_utils import shutdown_cleanup_thread
from seisLM.utils.config_utils import ConfigTracker

In [30]:
config_path = '/scicore/home/dokman0000/liu0003/projects/seisLM/seisLM/configs/pretrain/pretrain_config_layernorm_peak.json'

with open(config_path, "r", encoding="utf-8") as f:
  config = json.load(f)
config = ml_collections.ConfigDict(config)

DEFAULT_NUM_WORKERS = 8

print("Running in test mode")
config.training_config.max_epochs = 1
config.data_config.data_name = ['ETHZ']
config.data_config.local_batch_size = 8


tracked_config = ConfigTracker(config)

print(config.training_config.accelerator)

aa = tracked_config.get_unused_keys()
aa

Running in test mode
ddp


{'data_config.amp_norm',
 'data_config.amp_norm_type',
 'data_config.cache_dataset',
 'data_config.data_name',
 'data_config.demean',
 'data_config.local_batch_size',
 'data_config.prefetch_factor',
 'data_config.training_fraction',
 'model_config.activation_dropout',
 'model_config.apply_spec_augment',
 'model_config.attention_dropout',
 'model_config.codevector_dim',
 'model_config.contrastive_logits_temperature',
 'model_config.conv_bias',
 'model_config.conv_dim',
 'model_config.conv_kernel',
 'model_config.conv_stride',
 'model_config.diversity_loss_weight',
 'model_config.do_stable_layer_norm',
 'model_config.feat_extract_norm',
 'model_config.feat_proj_dropout',
 'model_config.feat_quantizer_dropout',
 'model_config.hidden_dropout',
 'model_config.hidden_size',
 'model_config.initializer_range',
 'model_config.input_dim',
 'model_config.intermediate_size',
 'model_config.layer_norm_eps',
 'model_config.layerdrop',
 'model_config.mask_feature_length',
 'model_config.mask_feature_

In [16]:
import ml_collections

config_dict = ml_collections.ConfigDict()
config_dict.a = 1
config_dict.b = ml_collections.ConfigDict()
config_dict.b.c = 2
config_dict.b.d = 3
config_dict.e = 4

# Wrap the config in the tracker
tracked_config = ConfigTracker(config_dict)

# Access some configuration fields
print(tracked_config.a)        # Access 'a'
print(tracked_config.b.c)      # Access 'b.c'

# Get unused fields
unused_keys = tracked_config.get_unused_keys()
print("Unused keys:", unused_keys)
print(config_dict)


1
2
Unused keys: {'b.c', 'b.d', 'e'}
a: 1
b:
  c: 2
  d: 3
e: 4

